# Выбор локации для скважины

# Choice of location for the oil well

- [Шаг 0. Описание проекта.](#Step_1)
- [Шаг 1. Загрузка и подготовка данных.](#Step_2)
- [Шаг 2. Обучение и проверка модели.](#Step_3)<br />
[Пункт A. Обучение модели.](#Step_4) <br />
[Пункт B. Анализ модели.](#Step_5) <br />
- [Шаг 3. Подготовка к расчету прибыли.](#Step_6)
- [Шаг 4. Расчет прибыли и рисков.](#Step_7) 
[Пункт A. Регион 0.](#Step_8) <br />
[Пункт B. Регион 1.](#Step_9) <br />
[Пункт C. Регион 2.](#Step_10) <br />

- [Step 0. Description of the project.](#Step_1)
- [Step 1. Loading and preparing data.](#Step_2)
- [Step 2. Train and validate the model.](#Step_3)<br />
[Item A. Training the model.](#Step_4) <br />
[Point B. Model analysis.](#Step_5) <br />
- [Step 3. Preparing for profit calculation.](#Step_6)
- [Step 4. Profit and risk calculation.](#Step_7)
[Item A. Region 0.](#Step_8) <br />
[Item B. Region 1.](#Step_9) <br />
[Item C. Region 2.](#Step_10) <br />

<a id='Step_1'></a>
# Описание проекта

# Description of the project

Перед нами стоит задача - выбрать место для бурения новой нефтяной скважины.

Нам доступны пробы нефти из трёх регионов, каждый из которых содержит 10000 месторождений. По каждому месторождению известны качество нефти и объём запасов. Необходимо разработать модель машинного обучения сопосбную определить регион, который принесет наибольшую прибыль. Также нужно проанализировать ожидаемую прибыль и риски при помощи Bootstrap.

Порядок выбора места:
- В выбранном регионе ищут месторождения и для каждого из них определяют значения признаков.
- Разрабатывают модель и оценивают количество запасов.
- Выбирают места с наивысшими оценками параметров. Количество месторождений зависит от доступного бюджета и стоимости разработки месторождения.
- Считают прибыль, как сумму прибыли выбранным месторождений.

Our task is to choose a place for drilling a new oil well.

We have access to oil samples from three regions, each containing 10,000 deposits. For each field, the quality of oil and the volume of reserves are known. It is necessary to develop a machine learning model capable of identifying the region that will bring the greatest profit. You also need to analyze the expected profit and risks using Bootstrap.

Place selection procedure:
- In the selected region, deposits are searched and for each of them the values ​​of the features are determined.
- Develop a model and estimate the amount of reserves.
- Choose places with the highest parameter scores. The number of deposits depends on the available budget and the cost of developing the deposit.
- Calculate the profit as the sum of the profits of the selected deposits.

<a id='Step_2'></a>
# 1. Загрузка и подготовка данных

# 1. Loading and preparing data

Загрузка библиотек.

Loading libraries.

In [ ]:
!pip install statsmodels==0.12.1
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from statsmodels.stats import outliers_influence as oi
from statsmodels.stats import diagnostic as dc
from scipy import stats as st
import statsmodels

Загрузка данных.

Loading data.

In [ ]:
df0=pd.read_csv('/datasets/geo_data_0.csv')
df1=pd.read_csv('/datasets/geo_data_1.csv')
df2=pd.read_csv('/datasets/geo_data_2.csv')

Общий отчет.

General report.

In [ ]:
for doc in (df0, df1, df2):
    print("First 5")
    print(doc.head())
    print()
    print("Info")
    print(doc.info())
    print()
    print("Duplicates")
    print(doc.duplicated().sum())
    print()
    print("########################################")

First 5
      id        f0        f1        f2     product
0  txEyH  0.705745 -0.497823  1.221170  105.280062
1  2acmU  1.334711 -0.340164  4.365080   73.037750
2  409Wp  1.022732  0.151990  1.419926   85.265647
3  iJLyR -0.032172  0.139033  2.978566  168.620776
4  Xdl7t  1.988431  0.155413  4.751769  154.036647

Info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None

Duplicates
0

########################################
First 5
      id         f0         f1        f2     product
0  kBEdx -15.001348  -8.276000 -0.005876    3.179103
1  62mP7  14.272088  -3.475083  0.999183   26.953261
2  vyE1P   6.263187  -5.948386  5.001160  134.766305
3  KcrkZ -13.081196 -11.506057  4.999415  137.945408
4  AHL4O  1

In [ ]:
print(df0.head())
print(df1.head())
print(df2.head())

      id        f0        f1        f2     product
0  txEyH  0.705745 -0.497823  1.221170  105.280062
1  2acmU  1.334711 -0.340164  4.365080   73.037750
2  409Wp  1.022732  0.151990  1.419926   85.265647
3  iJLyR -0.032172  0.139033  2.978566  168.620776
4  Xdl7t  1.988431  0.155413  4.751769  154.036647
      id         f0         f1        f2     product
0  kBEdx -15.001348  -8.276000 -0.005876    3.179103
1  62mP7  14.272088  -3.475083  0.999183   26.953261
2  vyE1P   6.263187  -5.948386  5.001160  134.766305
3  KcrkZ -13.081196 -11.506057  4.999415  137.945408
4  AHL4O  12.702195  -8.147433  5.004363  134.766305
      id        f0        f1        f2     product
0  fwXo0 -1.146987  0.963328 -0.828965   27.758673
1  WJtFt  0.262778  0.269839 -2.530187   56.069697
2  ovLUW  0.194587  0.289035 -5.586433   62.871910
3  q6cA6  2.236060 -0.553760  0.930038  114.572842
4  WPMUX -0.515993  1.716266  5.899011  149.600746


### Summary.
- Данные по каждому региону (регион 0 = df0, регион 1 = df1, регион 2 = df2) содержатся раздельно в соотетсвующих файлах.
- Каждый регион представлен 100000 наблюдений. Пропусков и дубликатов не обнаружено.
- Переменные хранятся в подходящих форматах.

### Summary.
- Data for each region (region 0 = df0, region 1 = df1, region 2 = df2) are contained separately in their respective files.
- Each region is represented by 100,000 observations. No gaps or duplicates found.
- Variables are stored in suitable formats.

<a id='Step_3'></a>
# 2. Обучение и проверка модели

#2 Train and validate the model

<a id='Step_4'></a>
### Обучение модели.

### Model training.

In [ ]:
# Список для предсказаний.
predictions_values= []
# Список для средних предсказаний.
predictions_means = []
# Список для средних ошибок прогнозирования
RMSE = []
# Список для коэфициентов
coefs = []
# Список для R2.
scores = []
# Цикл для обучения моделй и получения предсказаний по всем трем регионам.
for doc in (df0, df1, df2):
    # Сохраним признаки.
    features = doc[['f0', 'f1', 'f2']]
    # сохраним целевую перменную.
    target = doc['product']
    # Разделем данные на обучение и валидацию.
    features_train, features_valid, target_train, target_valid = train_test_split(features, target, 
                                                                                  test_size=0.25, random_state=42)
    # Обучим модель. 
    reg = LinearRegression().fit(features_train, target_train)
    # Получим предсказания
    prediction = reg.predict(features_valid)
    # Получим средние ошибки.
    rmse = mean_squared_error(target_valid, prediction) ** 0.5
    # Добавим предсказания в спискок.
    predictions_values.append(prediction)
    # Добавим средние предсказания в список.к
    predictions_means.append(prediction.mean())
    # Добавим средние ошибки в спискок. 
    RMSE.append(rmse)
    # Добавим R2 в список.
    scores.append(reg.score(features_train, target_train))
    # Сделаем ДФ для средних, ошибок и R2.
    reg_table = pd.DataFrame({"Avg_pred_product" : predictions_means, "RMSE" : RMSE, "R2" : scores})
    # Добавим коэффициенты в список.
    coefs.append(list(reg.coef_))
    # Сделаем ДФ для коэффициентов.
    coefs_table =  pd.DataFrame(coefs, columns=['coef_f0', 'coef_f1', 'coef_f2'])
    # Напечатаем средние, ошибки и R2 а экране
reg_table

,Avg_pred_product,RMSE,R2
0,92.398800,37.756600,0.276653
1,68.712878,0.890280,0.999624
2,94.771024,40.145872,0.199613


Видим большой разброс и больше среднее у регионов 0 и 2. Маленький разброс и меньшее среднее у 1-ого региона. R2 слишком маленький у 0-ого и 2-ого региона.

We see a large scatter and a larger average for regions 0 and 2. A small scatter and a smaller average for the 1st region. R2 is too small for the 0th and 2nd regions.

<a id='Step_5'></a>
### Анализ модели.

### Analysis of the model.

#### Интерпритация коэфициентов.

#### Interpretation of coefficients.

В данной ситуации, коэффициенты отлично итерпретируются. Разберем на примере 1-ого региона.

In this situation, the coefficients are perfectly interpreted. Let's look at the example of the 1st region.

In [ ]:
print(coefs_table)

    coef_f0    coef_f1    coef_f2
0  3.832254 -14.260473   6.593573
1 -0.144913  -0.021672  26.953109
2 -0.084158  -0.015461   5.750120


- С увеличением f0 на 1 ед. product снижается на -0.144913.
- С увеличением f1 на 1 ед. product снижается на -0.021672.
- С увеличением f2 на 1 ед. product увеличивается на 26.953109.
- То есть в этом регионе наибольший вес из предствленных переменных имеет f2.
- Можно заметить, что для 0-ого региона больший вес имеет f1, а для 2-ого f2.

- With an increase in f0 by 1 unit. product goes down by -0.144913.
- With an increase in f1 by 1 unit. product goes down by -0.021672.
- With an increase in f2 by 1 unit. product increases by 26.953109.
- That is, in this region, f2 has the greatest weight of the presented variables.
- It can be seen that for the 0th region, f1 has more weight, and for the 2nd region, f2.

#### Сравнение результатов прогнозирования при помощи линейной регрессии с моделей среднего.

#### Comparison of prediction results using linear regression from mean models.

Попробуем в качестве прогноза использовать среднее.

Let's try to use the average as a forecast.

In [ ]:
predictions = []
RMSE = []
scores = []
for doc in (df0, df1, df2):
    features = doc[['f0', 'f1', 'f2']]
    target = doc['product']
    features_train, features_valid, target_train, target_valid = train_test_split(features, target, 
                                                                                  test_size=0.25, random_state=42)
    prediction = [target_train.mean()] * len(target_valid)
    rmse = mean_squared_error(target_valid, prediction) ** 0.5
    RMSE.append(rmse)
    scores.append(reg.score(features_train, target_train))
    reg_table = pd.DataFrame({"RMSE" : RMSE, "R2" : scores})
print(reg_table)


        RMSE        R2
0  44.277235  0.226749
1  45.944859  0.060432
2  44.782777  0.199613


Для 0-ого и 2-ого региона мало что изменилось, а вот для 1-ого региона линейная регрессия намного лучше просто среднего.

For the 0th and 2nd region, little has changed, but for the 1st region, the linear regression is much better than just the average.

#### Исследование на мультиколлинеарность. 

#### Research on multicollinearity.

Мультиколлениарность увеличивает стандартные ошибки. Что бы ее выявить используют коэффициент вздутия дисперсии (VIF), который считается как регрессия на j-ой перменной на остальные переменные. VIF больще 10 свидетельсвует о возможном наличии мультиколлинеарности.

Multicolleniality increases standard errors. To reveal it, the inflation factor of variance (VIF) is used, which is considered as a regression on the j-th variable on the remaining variables. VIF greater than 10 indicates the possible presence of multicollinearity.


Теоритических оснвоаний для подозорения переменных на мультиколлинеарность у меня нет, но я проверю ради опыта. 

I have no theoretical basis for suspecting variables for multicollinearity, but I will check for the sake of experience.

In [ ]:
print('VIF-ы для переменных 0-ого региона')
for exog_idx in range(0,3):
    print(oi.variance_inflation_factor(np.array(df0[['f0', 'f1', 'f2']]), exog_idx))
print("")
print('VIF-ы для переменных 1-ого региона')
for exog_idx in range(0,3):
    print(oi.variance_inflation_factor(np.array(df1[['f0', 'f1', 'f2']]), exog_idx))
print("")
print('VIF-ы для переменных 2-ого региона')
for exog_idx in range(0,3):
    print(oi.variance_inflation_factor(np.array(df2[['f0', 'f1', 'f2']]), exog_idx))

VIF-ы для переменных 0-ого региона
1.157011142815797
1.1359994455991602
1.231341538646144

VIF-ы для переменных 1-ого региона
1.027315592291686
1.4945909857898279
1.5075610902870005

VIF-ы для переменных 2-ого региона
1.0000003783447908
1.0000002823930898
1.0000001055140126


Вероятно, мультиколлинеарность отсутсвут: все VIF-ы меньше 10.

There is probably no multicollinearity: all VIFs are less than 10.

### Summary.
- Были построенны регрессии для трех регионов: удоволитворительный результат сточки зрения R2, был получен только для 1-ого.
- Наибольшие средние значения предсказанных значений целевой переменной находятся в 0-ом и 2-ом регионах.
- Наибольшие дисперсии предсказанных значений целевой переменной находятся в 0-ом и 2-ом регионах.
- 1-ый регион намнрго более предсказуем, но от него не приходится ждать больших значений целевой переменной.
- Регрессия сработала лучше модели средних.
- Коэффициенты были проинтерпретированны.
- Теоретических оснований мультиколлинеарности нет; подсчет VIF это подтверждает. 

### Summary.
- Regressions were built for three regions: a satisfactory result from the point of view of R2 was obtained only for the 1st.
- The largest average values ​​of the predicted values ​​of the target variable are in the 0th and 2nd regions.
- The largest variances of the predicted values ​​of the target variable are in the 0th and 2nd regions.
- The 1st region is much more predictable, but one should not expect large values ​​of the target variable from it.
- The regression performed better than the mean model.
- The coefficients have been interpreted.
- There are no theoretical grounds for multicollinearity; the VIF count confirms this.

<a id='Step_6'></a>
# 3. Подготовка к расчёту прибыли

# 3. Preparing to Calculate Profits

Добавим условия задачи в отдельные переменные. 

Let's add the task conditions to separate variables.

In [ ]:
budget = 10000000000
proceeds_per_barrel = 450000
fields_to_explore = 500
fields_to_develop = 200
lesion_proba_threshold = .025

Посчитаем минимальный объем сырья в регионе, необходимый для покрытия потраченного бюджета.

Let's calculate the minimum amount of raw materials in the region necessary to cover the spent budget.

In [ ]:
min_product_per_field = budget / proceeds_per_barrel / 200
print('Минимальный объем нефти в скважине для окупаемости затрат: {:.2f} баррелей.'.format(min_product_per_field))

Минимальный объем нефти в скважине для окупаемости затрат: 111.11 баррелей.


Minimum volume of oil in the well for cost recovery: 111.11 barrels.

In [ ]:
j = -1
for doc in (df0, df1, df2):
    j += 1
    print('Минимальный объем нефти в скважине для окупаемости затрат больше среднего запаса в регионе {} на {:.2f} баррелей.'
          .format((j), (min_product_per_field - doc['product'].mean())))

Минимальный объем нефти в скважине для окупаемости затрат больше среднего запаса в регионе 0 на 18.61 баррелей.
Минимальный объем нефти в скважине для окупаемости затрат больше среднего запаса в регионе 1 на 42.29 баррелей.
Минимальный объем нефти в скважине для окупаемости затрат больше среднего запаса в регионе 2 на 16.11 баррелей.


The minimum volume of oil in the well for cost recovery is 18.61 barrels more than the average reserve in region 0.
The minimum volume of oil in the well for cost recovery is 42.29 barrels more than the average reserve in region 1.
The minimum volume of oil in the well for cost recovery is 16.11 barrels more than the average reserve in region 2.

### Summary.
- Минимально в скважене должно быть 111.11 баррелей нефти.
- В среднем скважена в любом регионе убыточна - необходимо тщательно подбирать скважены.

### Summary.
- The minimum well should be 111.11 barrels of oil.
- On average, a well in any region is unprofitable - it is necessary to carefully select wells.

<a id='Step_7'></a>
# 4. Расчёт прибыли и рисков 

# 4. Profit and Risk Calculation

Напишем функцию для подсчета прибыли.

Let's write a function to calculate profit.

In [ ]:
def revenue(pred, target, proceeds_per_barrel, fields_to_develop):
    # Принимает на вход предсказания, фактические значения, цену за 1 тыс. баррелей, количество точек для разработки.
    # Выбираем количество точк для разработки с нивысшими значениями предсказаний.
    pred_sorted = pred.sort_values(ascending=False)[:fields_to_develop]
    # Получаем фактические значения целевого признака на этих точках.
    target_selected = target[pred_sorted.index]
    # Считаем сколько прибыли было бы получено.
    return proceeds_per_barrel * target_selected.sum() - budget

<a id='Step_8'></a>
### Регион 0.

In [ ]:
# Получим предсказания по всему региону. 
pred_df0 = pd.Series(reg.predict(df0[['f0', 'f1', 'f2']]))
# Установим псевдослучайность.
state = np.random.RandomState(12345)
# Создадим пустой список для сохранения значений прибыли.
values_df0 = []
# Цикл, который считает предпологаемую прибыль.
for i in range(1000):
    # Делаем будстреп 500 точек из всех точек в регионе.
    target_subsample_df0 = df0['product'].sample(fields_to_explore, random_state=state, replace=True)
    # Получаем предсказания, которые мы бы получили для этих точек. 
    pred_subsample_df0 = pred_df0[target_subsample_df0.index] 
    # Считаем прибыль.
    values_df0.append(revenue(pred_subsample_df0, target_subsample_df0, proceeds_per_barrel, fields_to_develop))
# Сохранияем в сериес для удобства.
values_df0 = pd.Series(values_df0)
# Считаем среднее прибыли.
mean_value_df0 = values_df0.mean()
print("Регион 0.")
print("Среднее значение прибыли:", mean_value_df0)
# Выводим 95% доверительный интервал.
lower_df0 = values_df0.quantile(0.025)
upper_df0 = values_df0.quantile(0.975)
print(lower_df0)
print(upper_df0)

Регион 0.
Среднее значение прибыли: 330492697.5888724
-212423247.87151995
909646913.7112546


Region 0.

Average Profit: 330492697.5888724

95% confidence interval:

-212423247.87151995

909646913.7112546

Посмотрим на риск получения убытка: выручка меньше 10 млрд.р..

Let's look at the risk of loss: revenue is less than 10 billion rubles.

In [ ]:
risk_df0 = (values_df0<0).sum()/len(values_df0)
print("Регион 0.")
print("Вероятность получить убыток:", risk_df0)
print("Максимально допустимая вероятность получить убыток:", lesion_proba_threshold)

Регион 0.
Вероятность получить убыток: 0.119
Максимально допустимая вероятность получить убыток: 0.025


Region 0.

Probability of getting a loss: 0.119

Maximum allowable probability of taking a loss: 0.025

Получили вероятность ниже пороговой. 

Недопустимый риск убытка обнаружен. 

We got a probability below the threshold.

An unacceptable risk of loss has been detected.

<a id='Step_9'></a>
### Регион 1.

### Region 1.

In [ ]:
# Получим предсказания по всему региону. 
pred_df1 = pd.Series(reg.predict(df1[['f0', 'f1', 'f2']]))
# Установим псевдослучайность.
state = np.random.RandomState(12345)
# Создадим пустой список для сохранения значений прибыли.
values_df1 = []
# Цикл, который считает предпологаемую прибыль.
for i in range(1000):
    # Делаем будстреп 500 точек из всех точек в регионе.
    target_subsample_df1 = df1['product'].sample(fields_to_explore, random_state=state, replace=True)
    # Получаем предсказания, которые мы бы получили для этих точек. 
    pred_subsample_df1 = pred_df1[target_subsample_df1.index] 
    # Считаем прибыль.
    values_df1.append(revenue(pred_subsample_df1, target_subsample_df1, proceeds_per_barrel, fields_to_develop))
# Сохранияем в сериес для удобства.
values_df1 = pd.Series(values_df1)
# Считаем среднее прибыли.
mean_value_df1 = values_df1.mean()
print("Регион 1.")
print("Среднее значение прибыли:", mean_value_df1)
# Выводим 95% доверительный интервал.
lower_df1 = values_df1.quantile(0.025)
upper_df1 = values_df1.quantile(0.975)
print(lower_df1)
print(upper_df1)

Регион 1.
Среднее значение прибыли: 498769860.17802864
73741917.31029607
896096796.3533735


Region 1.

Average Profit: 498769860.17802864

95% confidence interval:

73741917.31029607

896096796.3533735

Посмотрим на риск получения убытка (отрицательная прибыль).

Let's look at the risk of loss (negative profit).

In [ ]:
risk_df1 = (values_df1<0).sum()/len(values_df1)
print("Регион 1.")
print("Вероятность получить убыток:", risk_df1)
print("Максимально допустимая вероятность получить убыток:", lesion_proba_threshold)

Регион 1.
Вероятность получить убыток: 0.013
Максимально допустимая вероятность получить убыток: 0.025


Region 1.

Probability of making a loss: 0.013

Maximum allowable probability of taking a loss: 0.025

Риск убытка в пределах допустимого порога.

Risk of loss within acceptable threshold.

<a id='Step_10'></a>
### Регион 2.

### Region 2.

In [ ]:
# Получим предсказания по всему региону. 
pred_df2 = pd.Series(reg.predict(df2[['f0', 'f1', 'f2']]))
# Установим псевдослучайность.
state = np.random.RandomState(12345)
# Создадим пустой список для сохранения значений прибыли.
values_df2 = []
# Цикл, который считает предпологаемую прибыль.
for i in range(1000):
    # Делаем будстреп 500 точек из всех точек в регионе.
    target_subsample_df2 = df2['product'].sample(fields_to_explore, random_state=state, replace=True)
    # Получаем предсказания, которые мы бы получили для этих точек. 
    pred_subsample_df2 = pred_df2[target_subsample_df2.index] 
    # Считаем прибыль.
    values_df2.append(revenue(pred_subsample_df2, target_subsample_df2, proceeds_per_barrel, fields_to_develop))
# Сохранияем в сериес для удобства.
values_df2 = pd.Series(values_df2)
# Считаем среднее прибыли.
mean_value_df2 = values_df2.mean()
print("Регион 2.")
print("Среднее значение прибыли:", mean_value_df2)
# Выводим 95% доверительный интервал.
lower_df2 = values_df2.quantile(0.025)
upper_df2 = values_df2.quantile(0.975)
print(lower_df2)
print(upper_df2)

Регион 2.
Среднее значение прибыли: 425107170.35025847
-151353761.24355105
965553593.1619681


Region 2.

Average Profit: 425107170.35025847

-151353761.24355105

965553593.1619681

Посмотрим на риск получения убытка.

Let's look at the risk of loss.

In [ ]:
risk_df2 = (values_df2<0).sum()/len(values_df2)
print("Регион 2.")
print("Вероятность получить убыток:", risk_df2)
print("Максимально допустимая вероятность получить убыток:", lesion_proba_threshold)

Регион 2.
Вероятность получить убыток: 0.078
Максимально допустимая вероятность получить убыток: 0.025


Region 2.

Probability of getting a loss: 0.078

Maximum allowable probability of taking a loss: 0.025

Недопустимый риск убытка обнаружен. 

An unacceptable risk of loss has been detected.

## Сведем результаты в таблицу.

## Let's summarize the results in a table.

In [ ]:
lower = [lower_df0, lower_df1, lower_df2]
upper = [upper_df0, upper_df1, upper_df2]
avg_profit = [values_df0.mean(), values_df1.mean(), values_df2.mean()]
risk = [risk_df0, risk_df1, risk_df2]
result_df = pd.DataFrame({"lower_int" : lower, "upper_int" : upper, "avg_profit" : avg_profit, "risk" : risk})
result_df["is_over_threshold"] = result_df['risk'].apply(lambda x: 1 if x > lesion_proba_threshold else 0) 
result_df

,lower_int,upper_int,avg_profit,risk,is_over_threshold
0,-2.124232e+08,9.096469e+08,3.304927e+08,0.119,1
1,7.374192e+07,8.960968e+08,4.987699e+08,0.013,0
2,-1.513538e+08,9.655536e+08,4.251072e+08,0.078,1


## Общий вывод.
- Была подготовлена модель регрессии для получения предсказаний целеовго признака по трем перменным.
- При при помощи модели и техники будстреп была определена средняя ожидаемая прибыль для каждого региона, а также доверительный интервал и риск получения убытка.
- Риск получения убытка оказался ниже порога только в регионе 1 со средней ожидаемой прибылью 498769860 рублей.

## General conclusion.
- A regression model was prepared to obtain target attribute predictions for three variables.
- Using the bootstrap model and technique, the average expected profit for each region was determined, as well as the confidence interval and the risk of loss.
- The risk of loss was below the threshold only in region 1 with an average expected profit of 498769860 rubles.